#### 支持向量机算法：  
#### 线性可分支持向量机学习算法——最大间隔法  
**输入**：线性可分训练数据集$T=\{(x_1,y_1),(x_2,y_2),...,(x_N,y_N) \}$，其中，$x_i\in X=R^n$，$y_i\in Y=\{-1,+1 \}$，$i=1,2,...,N$；  
**输出**：最大间隔分离超平面和分类决策函数。  
（1）构造并求解约束最优化问题：  
&emsp;&emsp;&emsp;${\underset{w,b}{min}}\frac{1}{2}||w||^2 $  
&emsp;&emsp;&emsp;$s.t.$&emsp;$y_i(w\cdot x+b)-1\ge 0$，$i=1,2,...N$  
&emsp;求解最优解$w^*$，$b^*$。  
（2）由此得到分离超平面：  
&emsp;&emsp;&emsp;$w\cdot x+b^*=0$  
&emsp;分类决策函数  
&emsp;&emsp;&emsp;$f(x)=sign(w^*\cdot x+b^*)$

In [1]:
import math
import random
import numpy as np

#### 加载文件  
$param fileName$:要加载的文件路径  
*return*: 数据集和标签集

In [2]:
def loadData(fileName):
    dataArr = []; labelArr = []
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split(',')
        dataArr.append([int(num) / 255 for num in curLine[1:]])
        if int(curLine[0]) == 0:
            labelArr.append(1)
        else:
            labelArr.append(-1)
            
    #返回数据集和标记
    return dataArr, labelArr

#### 定义支持向量机类  
$param trainDataList$:训练数据集  
$param trainLabelList$: 训练测试集  
$param sigma$: 高斯核中分母的$\sigma$  
$param C$:软间隔中的惩罚参数      
$param toler$:松弛变量  
$param i$:$\alpha$的下标  
$param testDataList$:测试数据集  
$param testLabelList$: 测试标签集

In [3]:
class SVM:
    # SVM相关参数初始化
    def __init__(self, trainDataList, trainLabelList, sigma = 10, C = 200, toler = 0.001):
        self.trainDataMat = np.mat(trainDataList) 
        self.trainLabelMat = np.mat(trainLabelList).T 

        self.m, self.n = np.shape(self.trainDataMat)
        self.sigma = sigma 
        self.C = C                           
        self.toler = toler                       

        self.k = self.calcKernel()             
        self.b = 0                              
        self.alpha = [0] * self.trainDataMat.shape[0] 
        self.E = [0 * self.trainLabelMat[i, 0] for i in range(self.trainLabelMat.shape[0])]
        self.supportVecIndex = []


    # 计算核函数
    def calcKernel(self):
        k = [[0 for i in range(self.m)] for j in range(self.m)]
        #循环遍历Xi
        for i in range(self.m):
            if i % 100 == 0:
                print('construct the kernel:', i, self.m)
            X = self.trainDataMat[i, :]
            for j in range(i, self.m):
                Z = self.trainDataMat[j, :]
                result = (X - Z) * (X - Z).T
                #分子除以分母后去指数，得到的即为高斯核结果
                result = np.exp(-1 * result / (2 * self.sigma**2))
                #将Xi*Xj的结果存放入k[i][j]和k[j][i]中
                k[i][j] = result
                k[j][i] = result
                
        #返回高斯核矩阵
        return k

    # 查看第i个α是否满足KKT条件
    def isSatisfyKKT(self, i):
        gxi =self.calc_gxi(i)
        yi = self.trainLabelMat[i]

        if (math.fabs(self.alpha[i]) < self.toler) and (yi * gxi >= 1):
            return True

        elif (math.fabs(self.alpha[i] - self.C) < self.toler) and (yi * gxi <= 1):
            return True

        elif (self.alpha[i] > -self.toler) and (self.alpha[i] < (self.C + self.toler)) \
                and (math.fabs(yi * gxi - 1) < self.toler):
            return True

        return False

    # 计算g(xi)
    def calc_gxi(self, i):
        gxi = 0
        index = [i for i, alpha in enumerate(self.alpha) if alpha != 0]
        #遍历每一个非零α
        for j in index:
            gxi += self.alpha[j] * self.trainLabelMat[j] * self.k[j][i]
        #求和结束后再单独加上偏置b
        gxi += self.b

        return gxi

    # 计算Ei
    def calcEi(self, i):
        gxi = self.calc_gxi(i)

        return gxi - self.trainLabelMat[i]

    # SMO中选择第二个变量
    def getAlphaJ(self, E1, i):
        E2 = 0
        maxE1_E2 = -1
        maxIndex = -1

        #获得Ei非0的对应索引组成的列表，列表内容为非0Ei的下标i
        nozeroE = [i for i, Ei in enumerate(self.E) if Ei != 0]
        #对每个非零Ei的下标i进行遍历
        for j in nozeroE:
            E2_tmp = self.calcEi(j)
            if math.fabs(E1 - E2_tmp) > maxE1_E2:
                maxE1_E2 = math.fabs(E1 - E2_tmp)
                E2 = E2_tmp
                maxIndex = j
        if maxIndex == -1:
            maxIndex = i
            while maxIndex == i:
                maxIndex = int(random.uniform(0, self.m))
            E2 = self.calcEi(maxIndex)
            
        #返回第二个变量的E2值以及其索引
        return E2, maxIndex

    def train(self, iter = 100):
        iterStep = 0; parameterChanged = 1

        while (iterStep < iter) and (parameterChanged > 0):
            print('iter:%d:%d'%( iterStep, iter))
            iterStep += 1
            parameterChanged = 0

            #循环遍历所有样本，用于找SMO中第一个变量
            for i in range(self.m):
                if self.isSatisfyKKT(i) == False:
                    E1 = self.calcEi(i)
                    E2, j = self.getAlphaJ(E1, i)

                    y1 = self.trainLabelMat[i]
                    y2 = self.trainLabelMat[j]
  
                    alphaOld_1 = self.alpha[i]
                    alphaOld_2 = self.alpha[j]

                    if y1 != y2:
                        L = max(0, alphaOld_2 - alphaOld_1)
                        H = min(self.C, self.C + alphaOld_2 - alphaOld_1)
                    else:
                        L = max(0, alphaOld_2 + alphaOld_1 - self.C)
                        H = min(self.C, alphaOld_2 + alphaOld_1)

                    if L == H:   continue
                    #计算α的新值
                    k11 = self.k[i][i]
                    k22 = self.k[j][j]
                    k21 = self.k[j][i]
                    k12 = self.k[i][j]
                    alphaNew_2 = alphaOld_2 + y2 * (E1 - E2) / (k11 + k22 - 2 * k12)
                    if alphaNew_2 < L:
                        alphaNew_2 = L
                    elif alphaNew_2 > H:
                        alphaNew_2 = H
                    alphaNew_1 = alphaOld_1 + y1 * y2 * (alphaOld_2 - alphaNew_2)

                    b1New = -1 * E1 - y1 * k11 * (alphaNew_1 - alphaOld_1) \
                            - y2 * k21 * (alphaNew_2 - alphaOld_2) + self.b
                    b2New = -1 * E2 - y1 * k12 * (alphaNew_1 - alphaOld_1) \
                            - y2 * k22 * (alphaNew_2 - alphaOld_2) + self.b

                    if (alphaNew_1 > 0) and (alphaNew_1 < self.C):
                        bNew = b1New
                    elif (alphaNew_2 > 0) and (alphaNew_2 < self.C):
                        bNew = b2New
                    else:
                        bNew = (b1New + b2New) / 2

                    self.alpha[i] = alphaNew_1
                    self.alpha[j] = alphaNew_2
                    self.b = bNew

                    self.E[i] = self.calcEi(i)
                    self.E[j] = self.calcEi(j)

                    if math.fabs(alphaNew_2 - alphaOld_2) >= 0.00001:
                        parameterChanged += 1

                print("iter: %d i:%d, pairs changed %d" % (iterStep, i, parameterChanged))

        # 全部计算结束后，重新遍历一遍α，查找里面的支持向量
        for i in range(self.m):
            if self.alpha[i] > 0:
                # 将支持向量的索引保存起来
                self.supportVecIndex.append(i)

    # 单独计算核函数
    def calcSinglKernel(self, x1, x2):
        result = (x1 - x2) * (x1 - x2).T
        result = np.exp(-1 * result / (2 * self.sigma ** 2))
        
        #返回结果
        return np.exp(result)


    # 对样本的标签进行预测
    def predict(self, x):
        result = 0
        for i in self.supportVecIndex:
            tmp = self.calcSinglKernel(self.trainDataMat[i, :], np.mat(x))
            result += self.alpha[i] * self.trainLabelMat[i] * tmp
        #求和项计算结束后加上偏置b
        result += self.b
        #使用sign函数返回预测结果
        return np.sign(result)


    # 测试
    def test(self, testDataList, testLabelList):
        errorCnt = 0
        for i in range(len(testDataList)):
            print('test:%d:%d'%(i, len(testDataList)))
            result = self.predict(testDataList[i])
            if result != testLabelList[i]:
                errorCnt += 1
                
        #返回正确率
        return 1 - errorCnt / len(testDataList)

In [6]:
# 获取训练集及标签
trainDataList, trainLabelList = loadData('Mnist/mnist_train.csv')

# 获取测试集及标签
testDataList, testLabelList = loadData('Mnist/mnist_test.csv')

#初始化SVM类
print('start init SVM')
svm = SVM(trainDataList[:1000], trainLabelList[:1000], 10, 200, 0.001)

# 开始训练
print('start to train')
svm.train()

# 开始测试
print('start to test')
accuracy = svm.test(testDataList[:100], testLabelList[:100])
print('the accuracy is:',accuracy)

start init SVM
construct the kernel: 0 1000
construct the kernel: 100 1000
construct the kernel: 200 1000
construct the kernel: 300 1000
construct the kernel: 400 1000
construct the kernel: 500 1000
construct the kernel: 600 1000
construct the kernel: 700 1000
construct the kernel: 800 1000
construct the kernel: 900 1000
start to train
iter:0:100
iter: 1 i:1, pairs changed 1
iter: 1 i:21, pairs changed 2
iter: 1 i:22, pairs changed 3
iter: 1 i:23, pairs changed 4
iter: 1 i:24, pairs changed 5
iter: 1 i:25, pairs changed 5
iter: 1 i:26, pairs changed 5
iter: 1 i:27, pairs changed 5
iter: 1 i:28, pairs changed 6
iter: 1 i:29, pairs changed 6
iter: 1 i:30, pairs changed 6
iter: 1 i:31, pairs changed 6
iter: 1 i:33, pairs changed 6
iter: 1 i:34, pairs changed 7
iter: 1 i:35, pairs changed 8
iter: 1 i:36, pairs changed 9
iter: 1 i:37, pairs changed 10
iter: 1 i:40, pairs changed 10
iter: 1 i:43, pairs changed 10
iter: 1 i:51, pairs changed 11
iter: 1 i:56, pairs changed 12
iter: 1 i:57, pai

iter: 2 i:23, pairs changed 0
iter: 2 i:24, pairs changed 0
iter: 2 i:26, pairs changed 0
iter: 2 i:28, pairs changed 0
iter: 2 i:29, pairs changed 0
iter: 2 i:30, pairs changed 0
iter: 2 i:31, pairs changed 0
iter: 2 i:32, pairs changed 0
iter: 2 i:33, pairs changed 0
iter: 2 i:34, pairs changed 0
iter: 2 i:35, pairs changed 0
iter: 2 i:36, pairs changed 0
iter: 2 i:37, pairs changed 0
iter: 2 i:38, pairs changed 0
iter: 2 i:40, pairs changed 0
iter: 2 i:41, pairs changed 0
iter: 2 i:42, pairs changed 0
iter: 2 i:43, pairs changed 0
iter: 2 i:44, pairs changed 0
iter: 2 i:45, pairs changed 0
iter: 2 i:46, pairs changed 0
iter: 2 i:51, pairs changed 0
iter: 2 i:53, pairs changed 0
iter: 2 i:54, pairs changed 0
iter: 2 i:55, pairs changed 0
iter: 2 i:56, pairs changed 0
iter: 2 i:57, pairs changed 0
iter: 2 i:58, pairs changed 0
iter: 2 i:61, pairs changed 0
iter: 2 i:63, pairs changed 0
iter: 2 i:67, pairs changed 0
iter: 2 i:68, pairs changed 0
iter: 2 i:70, pairs changed 0
iter: 2 i: